In [1]:
from SnakeBoard import SnakeBoard
from SnakeGame import SnakeGame
from NeuralNetwork import NeuralNetwork
import numpy as np
import time
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.3, Python 3.9.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# ---------- User defined parameters ----------
# Miscellaneous parameters
restore_weights_prev_training = 0
manual_play = 0 # Get user input (keyboard) instead of neural network auto-play
show_visuals = 0 # Show the games (1) or just play and calculated in the back-end (0)
t_between_gen = 0 # Time (secs) between generations
n_gens_2_save_weights = 25 # Num of generations elapsed to save weights in a file

# Training parameters
n_of_gens = 5000 # Number of training generations
n_games_per_gen = 5000 # Number of parallel games per generation
selected_games_per_gen = 25 # Selected baselines per generation to be used as references for mutations

# Mutation parameters
mrate_bias = 0.1
mrate_weights = 0.1
msize_bias = 0.1
msize_weights = 0.1

In [3]:
# ---------- Machine Learning main logic ---------- 
# Restore weights from previous training if required
if restore_weights_prev_training == 1:
    restored_data = np.load("./training_history.npy", allow_pickle=True)

# Create 'N' games + their ANN instances
record_score, record_w_score = 0, 0
s_board = SnakeBoard(n_games_per_gen)
s_games, s_ann = [] , []
for i in range(n_games_per_gen):
    s_games.append(SnakeGame(s_board))
    s_ann.append(NeuralNetwork())
    if restore_weights_prev_training == 1:
        s_ann[-1].set_weights_biases(restored_data[0][-1].weights, restored_data[0][-1].biases)

# Create history of the best scores and ANN weights
best_score_history, best_ann_weights_history = list(), list()
if restore_weights_prev_training == 1:
    best_score_history = restored_data[1]
    best_ann_weights_history = restored_data[0]

s_board.init_board()

# Run number of generations
for idx_gen in range(n_of_gens):
    # 1 Step all games in current generation (until all games are over)
    while True:     
        game_status = list() 
        # 1_1 Go into each game to step individually
        for idx_game, game in enumerate(s_games):
            
            #1_1_1 Get current game state and decide the next move
            state = game.get_game_state()
            if manual_play == 1:
                next_move = game.get_key()
            else:
                next_move = s_ann[idx_game].calculate(state) 
                if next_move == 0: next_move = "IDLE"
                elif next_move == 1: next_move = "T_LEFT"
                elif next_move == 2: next_move = "T_RIGHT"

            #1_1_2 Step game instance based on ANN calc. next move
            [game_over, w_score, score] = game.step_game(next_move)

            #1_1_3 Save game data in game status dictionary array
            game_status.append({"game_over":    game_over, 
                                "score":        score, 
                                "w_score":      w_score, 
                                "idx_game":     idx_game})       

        # 1_2 Update graphics of all games (visual feedback)
        if show_visuals == 1:
            s_board.clear_board()
            s_board.update_board_elements(s_games)

        # 1_3 If all game instances are over, finish current generation
        if np.min([g["game_over"] for g in game_status])==True:
            break
    
    # 2 Get best score + ANN in prev. generation
    game_status.sort(key=lambda x:x["w_score"],reverse =True) # Sort from best to worst game
    game_status_best = game_status[0]
    best_score_history = np.append(best_score_history,game_status_best["score"])
    best_ann_weights_history = np.append(best_ann_weights_history,s_ann[game_status_best["idx_game"]])
    print("GEN ", idx_gen, " --- BEST SCORE: ", 
            game_status_best["score"] , " / ", 
            game_status_best["w_score"] , " --- RECORD: ",
            record_score, " / ", record_w_score)

    # 3 If best score in curr. gen is an all-time record, save it
    if game_status_best["w_score"] > record_w_score:
        record_w_score = game_status_best["w_score"]
        record_score = game_status_best["score"]
    
    # 4 Get the best "selected_games_per_gen" games in the current generation
    # and place them in the first positions
    for i in range(selected_games_per_gen):
        s_ann[i] = s_ann[game_status[i]["idx_game"]].copy()
    
    # 5 Mutate the best ones in the subsequent positions
    for i in range(selected_games_per_gen, n_games_per_gen):
        s_ann[i] = s_ann[np.mod(i,selected_games_per_gen)].copy()
        s_ann[i].mutate(mrate_weights, msize_weights, mrate_bias, msize_bias) # random mutations

    # 6 Reset all games
    for idx_game, game in enumerate(s_games):
        game.reset_game()

    # 7 Save weights in an external file when applicable
    if np.mod(idx_gen,n_gens_2_save_weights)==0:
        np.save("./training_history.npy",[best_ann_weights_history, best_score_history])
    
    time.sleep(t_between_gen)

s_board.quit_board()

SnakeBoard instance created.
GEN  0  --- BEST SCORE:  17  /  535  --- RECORD:  0  /  0
GEN  1  --- BEST SCORE:  18  /  549  --- RECORD:  17  /  535
GEN  2  --- BEST SCORE:  22  /  700  --- RECORD:  18  /  549
GEN  3  --- BEST SCORE:  17  /  544  --- RECORD:  22  /  700
